In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os
from tqdm.notebook import tqdm
from shutil import copyfile
from skimage.metrics import structural_similarity as ssim

%matplotlib inline

In [2]:
def calculate_ssim(rgb1, rgb2) -> float:
    return ssim(
        cv2.cvtColor(rgb1, cv2.COLOR_RGB2YCrCb)[..., 0],
        cv2.cvtColor(rgb2, cv2.COLOR_RGB2YCrCb)[..., 0],
        data_range=255
    )

In [4]:
camera_type = 'iphone13'

devices_settings = {
    'iphone13': {
        'ext': '.TIF',
        'splitter': 'IMG_',
        'need_crop': True,
        'crop_box': [800, 800, 3000, 2200],
        'folder': 'iphone13',
        'stat': {
            'ISO': [100, 200, 400, 800, 3200],
            'shutter': [10, 20, 40, 80, 320]
        }
    },
    'sony': {
        'ext': '.tiff',
        'splitter': 'DSC',
        'need_crop': False,
        'folder': 'sonya7c',
        'stat': {
            'ISO': [250, 500, 2000, 4000, 8000, 16000],
            'shutter': [25, 50, 200, 400, 800, 1600]
        }
    }
}
        
settings = devices_settings[camera_type]

images_folder = '/media/alexey/HDDData/datasets/image_denoising/ours_dataset/temp_checking/{}/raw/'.format(
    settings['folder']
)
images_folder = '/media/alexey/HDDData/datasets/image_denoising/ours_dataset/sonya7c/scene_2/raw/'

images_paths = [
    os.path.join(images_folder, fname) 
    for fname in os.listdir(images_folder)
    if fname.endswith(settings['ext'])
]
    

images_paths.sort(key=lambda x: int(os.path.splitext(x)[0].split(settings['splitter'])[-1]))
images_paths = images_paths[:10]

if settings['need_crop']:
    print('Use crop')
    x1, y1, x2, y2 = settings['crop_box']
    
    images = [
        cv2.cvtColor(
            cv2.imread(img_path, cv2.IMREAD_COLOR)[y1:y2, x1:x2],
            cv2.COLOR_BGR2RGB
        )
        for img_path in tqdm(images_paths)
    ]
else:
    images = [
        cv2.cvtColor(cv2.imread(img_path, cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)
        for img_path in tqdm(images_paths)
    ]
    
images = np.array(images)

Use crop


0it [00:00, ?it/s]

In [6]:
base_path = '/media/alexey/HDDData/datasets/image_denoising/ours_dataset/iphone13/'
gts_path = '/media/alexey/HDDData/datasets/image_denoising/ours_dataset/aligned_frames/'

os.makedirs(os.path.join(gts_path, 'clear/'), exist_ok=True)
os.makedirs(os.path.join(gts_path, 'noisy/'), exist_ok=True)

for scene in tqdm(os.listdir(base_path)):
    images_folder = os.path.join(base_path, scene, 'raw')
    images_paths = [
        os.path.join(images_folder, fname) 
        for fname in os.listdir(images_folder)
        if fname.endswith(settings['ext'])
    ]
    
    images_paths.sort(key=lambda x: int(os.path.splitext(x)[0].split(settings['splitter'])[-1]))
    clear_images_paths = images_paths[:10]
    noisy_images_paths = images_paths[10:]
    
    if settings['need_crop']:
        x1, y1, x2, y2 = settings['crop_box']

        clear_images = [
            cv2.cvtColor(
                cv2.imread(img_path, cv2.IMREAD_COLOR)[y1:y2, x1:x2],
                cv2.COLOR_BGR2RGB
            )
            for img_path in clear_images_paths
        ]
    else:
        clear_images = [
            cv2.cvtColor(cv2.imread(img_path, cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)
            for img_path in clear_images_paths
        ]

    clear_images = np.array(clear_images)
    clear_frame = clear_images.astype(np.float32).mean(axis=0).astype(np.uint8)
    
    for i, noisy_img_path in enumerate(noisy_images_paths):
        gt_path = os.path.join(gts_path, 'clear/', 'custom_{}_{}_{}.tiff'.format(camera_type, scene, i + 1))
        noisy_path = os.path.join(gts_path, 'noisy/', 'custom_{}_{}_{}.tiff'.format(camera_type, scene, i + 1))
        
        assert cv2.imwrite(gt_path, cv2.cvtColor(clear_frame, cv2.COLOR_RGB2BGR)), gt_path
        
        _img = cv2.imread(noisy_img_path, cv2.IMREAD_COLOR)
        if settings['need_crop']:
            x1, y1, x2, y2 = settings['crop_box']
            _img = _img[y1:y2, x1:x2]
        
        assert cv2.imwrite(noisy_path, _img), noisy_path

  0%|          | 0/4 [00:00<?, ?it/s]

In [41]:
# Estimate PSNR, SIMM and deviations of these metrics


gts_path = '/media/alexey/HDDData/datasets/image_denoising/ours_dataset/aligned_frames/'
for camera_type in ['iphone13', 'sony']:
    settings = devices_settings[camera_type]
    base_path = '/media/alexey/HDDData/datasets/image_denoising/ours_dataset/{}/'.format(
        settings['folder']
    )
    
    clear_psnrs = []
    clear_ssims = []
    
    noisy_psnrs = [[] for _ in settings['stat']['ISO']]
    noisy_ssims = [[] for _ in settings['stat']['ISO']]
    
    for scene in tqdm(os.listdir(base_path)):
        images_folder = os.path.join(base_path, scene, 'raw')
        images_paths = [
            os.path.join(images_folder, fname) 
            for fname in os.listdir(images_folder)
            if fname.endswith(settings['ext'])
        ]

        images_paths.sort(key=lambda x: int(os.path.splitext(x)[0].split(settings['splitter'])[-1]))
        clear_images_paths = images_paths[:10]
        noisy_images_paths = images_paths[10:]

        if settings['need_crop']:
            # print('Use crop')
            x1, y1, x2, y2 = settings['crop_box']

            clear_images = [
                cv2.cvtColor(
                    cv2.imread(img_path, cv2.IMREAD_COLOR)[y1:y2, x1:x2],
                    cv2.COLOR_BGR2RGB
                )
                for img_path in clear_images_paths
            ]
        else:
            clear_images = [
                cv2.cvtColor(cv2.imread(img_path, cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)
                for img_path in clear_images_paths
            ]

        clear_images = np.array(clear_images)
        clear_frame = clear_images.astype(np.float32).mean(axis=0).astype(np.uint8)
        
        psnr_list = [cv2.PSNR(img, clear_frame) for img in clear_images]
        ssim_list = [calculate_ssim(img, clear_frame) for img in clear_images]
        
        clear_psnrs += psnr_list
        clear_ssims += ssim_list

        for i, noisy_img_path in enumerate(noisy_images_paths):
            _img = cv2.imread(noisy_img_path, cv2.IMREAD_COLOR)
            if settings['need_crop']:
                x1, y1, x2, y2 = settings['crop_box']
                _img = _img[y1:y2, x1:x2]
            
            _img = cv2.cvtColor(_img, cv2.COLOR_BGR2RGB)

            psnr = cv2.PSNR(_img, clear_frame)
            noisy_psnrs[i].append(psnr)
            
            ssim_metric = calculate_ssim(_img, clear_frame)
            noisy_ssims[i].append(ssim_metric)
            
    
    clear_psnrs = np.array(clear_psnrs)
    clear_ssims = np.array(clear_ssims)

    print(
        'CLEAR | {}: ISO: {}, S: 1/{}, E[PSNR]={:.2f}, STD[PSNR]={:.2f}, E[SSIM]={:.2f}, STD[SSIM]={:.2f}'.format(
            camera_type,
            settings['stat']['ISO'][0],
            settings['stat']['shutter'][0],
            clear_psnrs.mean(),
            clear_psnrs.std(),
            clear_ssims.mean(),
            clear_ssims.std()
        )
    )
    
    for i in range(len(noisy_psnrs)):
        if len(noisy_psnrs[i]) == 0:
            continue
        
        psnrs = np.array(noisy_psnrs[i])
        ssims = np.array(noisy_ssims[i])

        print(
            'NOISY | {}: ISO: {}, S: 1/{}, E[PSNR]={:.2f}, STD[PSNR]={:.2f}, E[SSIM]={:.2f}, STD[SSIM]={:.2f}'.format(
                camera_type,
                settings['stat']['ISO'][i + 1],
                settings['stat']['shutter'][i + 1],
                psnrs.mean(),
                psnrs.std(),
                ssims.mean(),
                ssims.std()
            )
        )

  0%|          | 0/4 [00:00<?, ?it/s]

CLEAR | iphone13: ISO: 100, S: 1/10, E[PSNR]=29.36, STD[PSNR]=4.66, E[SSIM]=0.86, STD[SSIM]=0.02
NOISY | iphone13: ISO: 200, S: 1/20, E[PSNR]=25.56, STD[PSNR]=1.78, E[SSIM]=0.83, STD[SSIM]=0.04
NOISY | iphone13: ISO: 400, S: 1/40, E[PSNR]=27.94, STD[PSNR]=4.20, E[SSIM]=0.82, STD[SSIM]=0.05
NOISY | iphone13: ISO: 800, S: 1/80, E[PSNR]=24.72, STD[PSNR]=1.80, E[SSIM]=0.91, STD[SSIM]=0.03


  0%|          | 0/8 [00:00<?, ?it/s]

CLEAR | sony: ISO: 250, S: 1/25, E[PSNR]=45.22, STD[PSNR]=1.21, E[SSIM]=0.98, STD[SSIM]=0.01
NOISY | sony: ISO: 500, S: 1/50, E[PSNR]=41.66, STD[PSNR]=1.40, E[SSIM]=0.95, STD[SSIM]=0.01
NOISY | sony: ISO: 2000, S: 1/200, E[PSNR]=37.02, STD[PSNR]=1.96, E[SSIM]=0.87, STD[SSIM]=0.04
NOISY | sony: ISO: 4000, S: 1/400, E[PSNR]=34.85, STD[PSNR]=1.82, E[SSIM]=0.81, STD[SSIM]=0.05
NOISY | sony: ISO: 8000, S: 1/800, E[PSNR]=34.12, STD[PSNR]=0.66, E[SSIM]=0.83, STD[SSIM]=0.04
NOISY | sony: ISO: 16000, S: 1/1600, E[PSNR]=30.53, STD[PSNR]=0.93, E[SSIM]=0.79, STD[SSIM]=0.02
